#Unity Catalog enforces centralized governance using a 3-level namespace:

Catalog


 └── Schema


           └── Table / View / Function


#Create Catalog & Schemas 

In [0]:
%sql
SELECT current_metastore();
SHOW CATALOGS;


In [0]:
%sql
CREATE CATALOG IF NOT EXISTS ecommerce;


In [0]:
%sql
USE CATALOG ecommerce;


In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS bronze;
CREATE SCHEMA IF NOT EXISTS silver;
CREATE SCHEMA IF NOT EXISTS gold;


In [0]:
%sql
SHOW SCHEMAS IN ecommerce;


Key Governance Takeaways (Don’t Skip)

✔ Catalog is not optional in Unity Catalog
✔ Schemas are permission boundaries
✔ Tables must belong to a schema
✔ This replaces the old db.schema.table pattern

#Managed vs External Tables

What is a Managed Table?

Databricks manages both metadata and data

Storage location is controlled by Unity Catalog

When you DROP TABLE, the data is deleted

What is an External Table?

Databricks manages metadata only

Data lives at a user-defined LOCATION

When you DROP TABLE, data remains

In [0]:
%sql
CREATE TABLE bronze.events
USING DELTA
AS
SELECT *
FROM read_files(
  '/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv',
  format => 'csv',
  header => 'true',
  inferSchema => 'true'
);


In [0]:
%sql
DESCRIBE EXTENDED bronze.events;




Look for:

Type: MANAGED

No custom location

In [0]:
%sql
CREATE TABLE silver.events
USING DELTA
AS
SELECT *
FROM bronze.events
WHERE event_type IS NOT NULL;


In [0]:
%sql
CREATE TABLE gold.products
USING DELTA
AS
SELECT
  product_id,
  brand,
  COUNT(*) AS purchases,
  SUM(price) AS revenue
FROM silver.events
WHERE event_type = 'purchase'
GROUP BY product_id, brand;


✔ All Managed Tables
✔ Fully governed by Unity Catalog
✔ Databricks-recommended design

In [0]:
CREATE TABLE bronze.events_external
USING DELTA
LOCATION 'xxxxxx'; -- Provide your S3 Storage Location and Register with UC

Shared S3/ADLS bucket

Data written by another system

Multi-engine access

#Access Control (GRANT / REVOKE)

How Access Control Works in Unity Catalog

Unity Catalog uses ANSI SQL GRANT / REVOKE with inheritance.

Permissions flow downward

More specific grants override broader ones

In [0]:
%sql
GRANT ALL PRIVILEGES ON SCHEMA ecommerce.silver TO `ram.katneni@gmail.com`;


In [0]:
%sql
GRANT SELECT ON TABLE ecommerce.gold.products TO `ram.katneni@gmail.com`;


In [0]:
%sql
SHOW GRANTS ON SCHEMA ecommerce.silver;
SHOW GRANTS ON SCHEMA ecommerce.gold;
SHOW GRANTS ON TABLE ecommerce.gold.products;


#Create Views for Controlled Access

Why Views in Unity Catalog?

Views act as a security boundary

Expose only required columns & rows

Preferred way to give analysts access to Gold data

Fully governed by Unity Catalog (permissions + lineage)

📌 Databricks Best Practice

Grant analysts access to views, not base tables.

In [0]:
%sql
CREATE OR REPLACE VIEW ecommerce.gold.top_products AS
SELECT
  product_id,
  brand,
  revenue,
  purchases
FROM ecommerce.gold.products
WHERE purchases > 10
ORDER BY revenue DESC
LIMIT 100;


✔ No raw fields
✔ Business-ready
✔ Safe for analysts

In [0]:
%sql
GRANT SELECT ON VIEW ecommerce.gold.top_products TO `ram.katneni@gmail.com`;


In [0]:
%sql
REVOKE SELECT ON TABLE ecommerce.gold.products FROM `ram.katneni@gmail.com`;


In [0]:
%sql
SHOW GRANTS ON VIEW ecommerce.gold.top_products;


#Data Lineage in Unity Catalog

![](/Volumes/ecommerce/default/vol/Image6.png)
![](/Volumes/ecommerce/default/vol/Image7.png)